In [76]:
import pandas as pd
import numpy as np
import requests
import json
from datetime import date
import configparser

In [8]:
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

In [10]:
# ENV = 'PROD'
ENV = 'DEV'

# familia = 'ooQKFaqwblGMPNq0AUML'
PROJECT = config[ENV]['PROJECT']
DB_API_KEY = config[ENV]['DB_API_KEY']
DB_PASSWORD = config[ENV]['DB_PASSWORD']
EMAIL = config[ENV]['EMAIL']

In [11]:
# Get the token
authUrl = f'https://identitytoolkit.googleapis.com/v1/accounts:signInWithPassword?key={DB_API_KEY}'

authData = {
    'email': EMAIL,
    'password': DB_PASSWORD,
    'returnSecureToken': True,
};

resp = requests.post(authUrl, data=json.dumps(authData))
if resp.status_code != 200:
    # This means something went wrong.
    raise BaseException('GET /tasks/ {}'.format(resp.status_code))
    
id_token = resp.json()['idToken']

In [13]:
# print(id_token)

In [21]:
headers = {'Authorization': f'Bearer {id_token}'}

In [14]:
# Vai getando
urlGet = f'https://firestore.googleapis.com/v1/projects/{PROJECT}/databases/(default)/documents/familias/'

resp = requests.get(urlGet, headers=headers)
if resp.status_code != 200:
    # This means something went wrong.
    raise BaseException('GET /tasks/ {}'.format(resp.status_code))

In [18]:
id_familias = []
json_familias = {}
for item in resp.json()['documents']:
    id_familia = item['name'].split('/')[-1]
    id_familias.append(id_familia)
    json_familias[id_familia] = {}
    
#     print('id da família: ', item['name'].split('/')[-1])
    
    for field in item:
        json_familias[id_familia][field] = item[field]
#         print(field, '->', item[field])

In [19]:
# print(id_familias)
# print(json_familias)

In [37]:
resps = {}
for id_familia in id_familias:
    urlGet = f'https://firestore.googleapis.com/v1/projects/{PROJECT}/databases/(default)/documents/familias/' + \
            f'{id_familia}/aniversariantes?pageSize=200'

    resp = requests.get(urlGet, headers=headers)
    
    if resp.status_code != 200:
        # This means something went wrong.
        raise BaseException('GET /tasks/ {}'.format(resp.status_code))
        
    resps[id_familia] = resp.json()['documents']

In [55]:
json_aniversariantes = {}
for id_familia, aniversariantes in resps.items():
    json_aniversariantes[id_familia] = {}
    
#     print(id_familia)
    
    for aniversariante in aniversariantes:
        id_aniversariante = aniversariante['name'].split('/')[-1]
        json_aniversariantes[id_familia][id_aniversariante] = aniversariante
        
#         print(id_aniversariante)
        
#         for field in aniversariante:
#             print(field, '->', aniversariante[field])

In [77]:
# print(json.dumps(json_aniversariantes, indent=2))

In [94]:
familias_ultimo_arquivo = 'familias_28-07-2020.json'

familias_ultimo_backup = {}

if familias_ultimo_arquivo:
    with open(familias_ultimo_arquivo) as json_file:
        familias_ultimo_backup = json.load(json_file)
    #     print(json.dumps(dados, indent=2))

In [95]:
aniversariantes_ultimo_arquivo = 'aniversariantes_28-07-2020.json'

aniversariantes_ultimo_backup = {}

if aniversariantes_ultimo_arquivo:
    with open(aniversariantes_ultimo_arquivo) as json_file:
        aniversariantes_ultimo_backup = json.load(json_file)
    #     print(json.dumps(dados, indent=2))

In [97]:
if json_familias != familias_ultimo_backup:

    filename = f'familias_{data_atual}.json'
    with open(filename, 'w') as outfile:
        json.dump(json_familias, outfile)
        print('Backup de famílias gerado')

In [98]:
if json_aniversariantes != aniversariantes_ultimo_backup:
    
    filename = f'aniversariantes_{data_atual}.json'
    with open(filename, 'w') as outfile:
        json.dump(json_aniversariantes, outfile)
        print('Backup de aniversariantes gerado')